# Importações de dados brutos

In [108]:
!pip install util-gfsilveira

In [109]:
import os, joblib, operator

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats

from util import *

meus_uteis()

-------------
-=< Lista >=-
-------------
0 -> printLinha()
1 -> printLis()
2 -> printDic()
3 -> timeProcess()
4 -> import_started()
5 -> img_save()
6 -> setup_img_save()
7 -> mask_corr_graphic()
8 -> structured_confusion()


In [110]:
dir = os.listdir()
printLis(dir)

-------------
-=< Lista >=-
-------------
0 -> .config
1 -> total_pergunta_wilcoxon.csv
2 -> total_pergunta_doc_wilcoxon.csv
3 -> perguntas.gz
4 -> total_pergunta_bio_wilcoxon.csv
5 -> df_mysql.gz
6 -> total_wilcoxon.csv
7 -> total_pergunta_gradua_wilcoxon.csv
8 -> total_pergunta_mestr_wilcoxon.csv
9 -> total_pergunta_exat_wilcoxon.csv
10 -> total_pergunta_posGradua_wilcoxon.csv
11 -> sample_data


In [111]:
perguntas = joblib.load(dir[3])
df = joblib.load(dir[5])
df.head()

,Data,IP,TCLE,Língua,Figura,Dintinguíveis,Proeminentes,Descrição,Eficiente,Útil,Estética,Fonte,Compreensão,Interesse,Grau,Área,Origem
id,,,,,,,,,,,,,,,,,
1,2020-03-02 13:31:20,200.134.9.2,s,pt,Reovírus Aviário,2,1,Sim,1,5,2,Sim,1,1,Doutorado,Exatas,Pesquisador
2,2020-03-02 17:45:41,191.52.71.228,s,pt,Reovírus Aviário,3,3,Sim,2,2,4,Não,4,2,Doutorado,Biológicas,Designer
3,2020-03-02 18:32:50,189.14.35.136,s,pt,Células Tronco,4,3,Sim,3,5,4,Não,3,3,Doutorado,Exatas,Pesquisador
4,2020-03-02 19:33:38,170.239.53.11,s,pt,Células Tronco,4,4,Sim,5,5,5,Sim,5,5,Mestrado,Biológicas,Designer
5,2020-03-03 11:50:20,189.103.158.150,s,pt,T. gondii,5,1,Sim,5,5,2,Sim,5,4,Doutorado,Biológicas,Pesquisador


# Funções

## Declaração

In [112]:
# custom func for confidence interval for differences 
# for (non-Gaussian unpaired data). Example: Mann-Whitney U test

def non_param_unpaired_CI(sample1, sample2, conf):
  n1 = len(sample1)  
  n2 = len(sample2)  
  alpha = 1-conf      
  N = stats.norm.ppf(1 - alpha/2) 

  # The confidence interval for the difference between the two population
  # medians is derived through the n x m differences.
  diffs = sorted([i-j for i in sample1 for j in sample2])
  
  # the Kth smallest to the Kth largest of the n x m differences then determine 
  # the confidence interval, where K is:
  k = np.math.ceil(n1*n2/2 - (N * (n1*n2*(n1+n2+1)/12)**0.5))
  
  CI = (round(diffs[k-1],4), round(diffs[len(diffs)-k],4))
  return CI

In [113]:
# custom func for confidence interval for differences 
# for (non-Gaussian paired data) Example: Wilcoxon signed-rank test

def non_param_paired_CI(sample1, sample2, conf):
  n = len(sample1)  
  alpha = 1-conf      
  N = stats.norm.ppf(1 - alpha/2) 

  # The confidence interval for the difference between the two population
  # medians is derived through the n(n+1)/2 possible averaged differences.
  diff_sample = sorted(list(map(operator.sub, sample2, sample1)))
  averages = sorted([(s1+s2)/2 for i, s1 in enumerate(diff_sample) for _, s2 in enumerate(diff_sample[i:])])

  # the Kth smallest to the Kth largest of the averaged differences then 
  # determine the confidence interval, where K is:
  k = np.math.ceil(n*(n+1)/4 - (N * (n*(n+1)*(2*n+1)/24)**0.5))

  CI = (round(averages[k-1],4), round(averages[len(averages)-k],4))
  return CI

In [114]:
def estates_table(pesquisador, designer):
  mean_pesquisador = np.mean(pesquisador).round(3)
  mean_designer = np.mean(designer).round(3)
  
  median_diff = np.median( list(map(operator.sub, designer, pesquisador)) )

  lower, upper = non_param_paired_CI(pesquisador, designer, 0.95)

  _, p = stats.wilcoxon(pesquisador, designer)
  if p < 0.001:
    p = "<0,001"
  else:
    p = f"{p.round(3)}".replace('.',',')
  
  return {
    'Média Pesquisador': mean_pesquisador,
    'Média Designer': mean_designer,
    "IC95% difer": f"{lower}-{upper}",
    "p-valor": p
  }

## Teste

In [115]:
# Mann-Whitney U test 
# for unpaired, non-normal continuous output data


# given
conf = 0.95
sample1 = [38,26,29,41,36,31,32,30,35,33]
sample2 = [45,28,27,38,40,42,39,39,34,45]

# mann-whitney
# we need to use alternative='two-sided'. The use_continuity param is set to 
# 'False' simply to compare with the findings of the book, 'Statistics at Square 
# One'. Otherwise, I think it can be left at its default value of 'True'.
stat, p = stats.mannwhitneyu(sample1, sample2, use_continuity=False, alternative='two-sided')
print(f"p: {round(p, 3)}")

if p<=(1-conf):
  print("Reject H0. Plasma globulin fraction looks likely to be affected by the treatments.")
else:
  print("Failed to reject H0. Plasma globulin fraction does not look likely to be affected by either treatments.")

# confidence interval
diff_median = np.median( list(map(operator.sub, sample1, sample2)) )
lower, upper = non_param_unpaired_CI(sample1, sample2, conf)

print(f"{100*conf} % confidence that the median of the samples' diff lies b/w {lower} and {upper}.")

print(f"Median of diff {diff_median}")

p: 0.075
Failed to reject H0. Plasma globulin fraction does not look likely to be affected by either treatments.
95.0 % confidence that the median of the samples' diff lies b/w -10 and 1.
Median of diff -5.5


In [116]:
# Wilcoxon rank test (used for paired data)

# (a) Ordinal output data or*
# (b) Quantitative output data (discrete or non-normal)

# given
conf = 0.95
before = [25,24,28,15,20,23,21,20,20,27]
after = [18,27,25,20,17,24,24,22,19,19]

# wilcoxon
statistic, p = stats.wilcoxon(before, after)
print(f"p: {round(p,3)}")

if p<=(1-conf):
  print("Reject H0. The sampling process seems to change fetal movements.")
else:
  print("Failed to reject H0. The sampling process does not seem to change fetal movements.")

# confidence interval
diff_median = np.median( list(map(operator.sub, before, after)) )
lower, upper = non_param_paired_CI(before, after, conf)

print(f"{100*conf} % confidence that the median of the samples' diff lies b/w {lower} and {upper}.")

print(f"Median of diff {diff_median}")

p: 0.681
Failed to reject H0. The sampling process does not seem to change fetal movements.
95.0 % confidence that the median of the samples' diff lies b/w -4.0 and 2.5.
Median of diff 0.0


In [117]:
estates_table(before, after)

{'IC95% difer': '-4.0-2.5',
 'Média Designer': 21.5,
 'Média Pesquisador': 22.3,
 'p-valor': '0,681'}

# Filtrando apenas perguntas

In [118]:
printLis(df.columns)

-------------
-=< Lista >=-
-------------
0 -> Data
1 -> IP
2 -> TCLE
3 -> Língua
4 -> Figura
5 -> Dintinguíveis
6 -> Proeminentes
7 -> Descrição
8 -> Eficiente
9 -> Útil
10 -> Estética
11 -> Fonte
12 -> Compreensão
13 -> Interesse
14 -> Grau
15 -> Área
16 -> Origem


In [119]:
columns = list(df.columns[[5,6,8,9,10,12,13,14,15,16]])
printLis(columns)

-------------
-=< Lista >=-
-------------
0 -> Dintinguíveis
1 -> Proeminentes
2 -> Eficiente
3 -> Útil
4 -> Estética
5 -> Compreensão
6 -> Interesse
7 -> Grau
8 -> Área
9 -> Origem


In [120]:
df_perg_quant = df[columns].copy()
df_perg_quant.head()

,Dintinguíveis,Proeminentes,Eficiente,Útil,Estética,Compreensão,Interesse,Grau,Área,Origem
id,,,,,,,,,,
1,2,1,1,5,2,1,1,Doutorado,Exatas,Pesquisador
2,3,3,2,2,4,4,2,Doutorado,Biológicas,Designer
3,4,3,3,5,4,3,3,Doutorado,Exatas,Pesquisador
4,4,4,5,5,5,5,5,Mestrado,Biológicas,Designer
5,5,1,5,5,2,5,4,Doutorado,Biológicas,Pesquisador


In [121]:
df_perg_quant.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 472 entries, 1 to 472
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Dintinguíveis  472 non-null    int32 
 1   Proeminentes   472 non-null    int32 
 2   Eficiente      472 non-null    int32 
 3   Útil           472 non-null    int32 
 4   Estética       472 non-null    int32 
 5   Compreensão    472 non-null    int32 
 6   Interesse      472 non-null    int32 
 7   Grau           472 non-null    object
 8   Área           472 non-null    object
 9   Origem         472 non-null    object
dtypes: int32(7), object(3)
memory usage: 27.7+ KB


# Estatística descritiva do Total de dados

In [122]:
pesq = []
for col in columns[:-3]:
  pesq += list(df_perg_quant[df_perg_quant.Origem == 'Pesquisador'][col])
len(pesq)

1652

In [123]:
desg = []
for col in columns[:-3]:
  desg += list(df_perg_quant[df_perg_quant.Origem != 'Pesquisador'][col])
len(desg)

1652

In [124]:
data = estates_table(pesq, desg)
data

{'IC95% difer': '0.5-1.0',
 'Média Designer': 4.147,
 'Média Pesquisador': 3.335,
 'p-valor': '<0,001'}

In [125]:
data = pd.DataFrame(data=data, index=['Total'], columns=data.keys())
data.to_csv('total_wilcoxon.csv')
data

,Média Pesquisador,Média Designer,IC95% difer,p-valor
Total,3.335,4.147,0.5-1.0,"<0,001"


# Estatística descritiva por pergunta

## Total

In [126]:
printLis(columns)

-------------
-=< Lista >=-
-------------
0 -> Dintinguíveis
1 -> Proeminentes
2 -> Eficiente
3 -> Útil
4 -> Estética
5 -> Compreensão
6 -> Interesse
7 -> Grau
8 -> Área
9 -> Origem


In [127]:
data = []
for col in columns[:-3]:
        data.append(estates_table(
            list(df_perg_quant[df_perg_quant.Origem == 'Pesquisador'][col]),
            list(df_perg_quant[df_perg_quant.Origem != 'Pesquisador'][col])
        ))
data

[{'IC95% difer': '0.5-1.0',
  'Média Designer': 4.085,
  'Média Pesquisador': 3.487,
  'p-valor': '<0,001'},
 {'IC95% difer': '0.5-1.0',
  'Média Designer': 4.047,
  'Média Pesquisador': 3.106,
  'p-valor': '<0,001'},
 {'IC95% difer': '0.5-1.0',
  'Média Designer': 3.864,
  'Média Pesquisador': 3.242,
  'p-valor': '<0,001'},
 {'IC95% difer': '0.0-0.0',
  'Média Designer': 4.712,
  'Média Pesquisador': 4.517,
  'p-valor': '0,022'},
 {'IC95% difer': '1.5-1.5',
  'Média Designer': 4.314,
  'Média Pesquisador': 2.822,
  'p-valor': '<0,001'},
 {'IC95% difer': '0.5-1.0',
  'Média Designer': 3.958,
  'Média Pesquisador': 3.242,
  'p-valor': '<0,001'},
 {'IC95% difer': '1.0-1.5',
  'Média Designer': 4.051,
  'Média Pesquisador': 2.928,
  'p-valor': '<0,001'}]

In [128]:
data = pd.DataFrame(data=data, index=columns[:-3], columns=data[0].keys())
data.to_csv('total_pergunta_wilcoxon.csv')
data

,Média Pesquisador,Média Designer,IC95% difer,p-valor
Dintinguíveis,3.487,4.085,0.5-1.0,"<0,001"
Proeminentes,3.106,4.047,0.5-1.0,"<0,001"
Eficiente,3.242,3.864,0.5-1.0,"<0,001"
Útil,4.517,4.712,0.0-0.0,"0,022"
Estética,2.822,4.314,1.5-1.5,"<0,001"
Compreensão,3.242,3.958,0.5-1.0,"<0,001"
Interesse,2.928,4.051,1.0-1.5,"<0,001"


## Área Biológicas

In [129]:
data = []
for col in columns[:-3]:
        data.append(estates_table(
            list(df_perg_quant[(df_perg_quant.Origem == 'Pesquisador') & (df_perg_quant.Área == 'Biológicas')].iloc[:-1][col]),
            list(df_perg_quant[(df_perg_quant.Origem != 'Pesquisador') & (df_perg_quant.Área == 'Biológicas')][col])
        ))
data

[{'IC95% difer': '0.5-1.0',
  'Média Designer': 4.09,
  'Média Pesquisador': 3.41,
  'p-valor': '<0,001'},
 {'IC95% difer': '1.0-1.5',
  'Média Designer': 4.039,
  'Média Pesquisador': 2.983,
  'p-valor': '<0,001'},
 {'IC95% difer': '0.5-1.0',
  'Média Designer': 3.86,
  'Média Pesquisador': 3.135,
  'p-valor': '<0,001'},
 {'IC95% difer': '0.0-0.0',
  'Média Designer': 4.725,
  'Média Pesquisador': 4.461,
  'p-valor': '0,008'},
 {'IC95% difer': '1.5-2.0',
  'Média Designer': 4.298,
  'Média Pesquisador': 2.618,
  'p-valor': '<0,001'},
 {'IC95% difer': '0.5-1.0',
  'Média Designer': 3.966,
  'Média Pesquisador': 3.124,
  'p-valor': '<0,001'},
 {'IC95% difer': '1.0-1.5',
  'Média Designer': 4.084,
  'Média Pesquisador': 2.798,
  'p-valor': '<0,001'}]

In [130]:
data = pd.DataFrame(data=data, index=columns[:-3], columns=data[0].keys())
data.to_csv('total_pergunta_bio_wilcoxon.csv')
data

,Média Pesquisador,Média Designer,IC95% difer,p-valor
Dintinguíveis,3.410,4.090,0.5-1.0,"<0,001"
Proeminentes,2.983,4.039,1.0-1.5,"<0,001"
Eficiente,3.135,3.860,0.5-1.0,"<0,001"
Útil,4.461,4.725,0.0-0.0,"0,008"
Estética,2.618,4.298,1.5-2.0,"<0,001"
Compreensão,3.124,3.966,0.5-1.0,"<0,001"
Interesse,2.798,4.084,1.0-1.5,"<0,001"


## Área Exatas

In [131]:
data = []
for col in columns[:-3]:
        data.append(estates_table(
            list(df_perg_quant[(df_perg_quant.Origem == 'Pesquisador') & (df_perg_quant.Área == 'Exatas')][col]),
            list(df_perg_quant[(df_perg_quant.Origem != 'Pesquisador') & (df_perg_quant.Área == 'Exatas')].iloc[:-11][col])
        ))
data

/usr/local/lib/python3.7/dist-packages/scipy/stats/morestats.py:2879: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


[{'IC95% difer': '0.5-2.0',
  'Média Designer': 4.391,
  'Média Pesquisador': 3.348,
  'p-valor': '0,011'},
 {'IC95% difer': '0.5-2.0',
  'Média Designer': 4.348,
  'Média Pesquisador': 3.0,
  'p-valor': '0,003'},
 {'IC95% difer': '0.0-1.5',
  'Média Designer': 3.957,
  'Média Pesquisador': 3.174,
  'p-valor': '0,069'},
 {'IC95% difer': '0.0-1.0',
  'Média Designer': 4.826,
  'Média Pesquisador': 4.478,
  'p-valor': '0,146'},
 {'IC95% difer': '1.0-2.5',
  'Média Designer': 4.522,
  'Média Pesquisador': 2.913,
  'p-valor': '<0,001'},
 {'IC95% difer': '0.5-2.0',
  'Média Designer': 4.304,
  'Média Pesquisador': 3.174,
  'p-valor': '0,008'},
 {'IC95% difer': '0.5-2.0',
  'Média Designer': 4.0,
  'Média Pesquisador': 2.957,
  'p-valor': '0,027'}]

In [132]:
data = pd.DataFrame(data=data, index=columns[:-3], columns=data[0].keys())
data.to_csv('total_pergunta_exat_wilcoxon.csv')
data

,Média Pesquisador,Média Designer,IC95% difer,p-valor
Dintinguíveis,3.348,4.391,0.5-2.0,"0,011"
Proeminentes,3.000,4.348,0.5-2.0,"0,003"
Eficiente,3.174,3.957,0.0-1.5,"0,069"
Útil,4.478,4.826,0.0-1.0,"0,146"
Estética,2.913,4.522,1.0-2.5,"<0,001"
Compreensão,3.174,4.304,0.5-2.0,"0,008"
Interesse,2.957,4.000,0.5-2.0,"0,027"


## Área Humanas

In [133]:
data = []
for col in columns[:-3]:
        data.append(estates_table(
            list(df_perg_quant[(df_perg_quant.Origem == 'Pesquisador') & (df_perg_quant.Área == 'Humanas')].iloc[:-10][col]),
            list(df_perg_quant[(df_perg_quant.Origem != 'Pesquisador') & (df_perg_quant.Área == 'Humanas')][col])
        ))
data

/usr/local/lib/python3.7/dist-packages/scipy/stats/morestats.py:2879: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


[{'IC95% difer': '-1.0-1.0',
  'Média Designer': 3.833,
  'Média Pesquisador': 3.833,
  'p-valor': '0,924'},
 {'IC95% difer': '-0.5-1.0',
  'Média Designer': 3.875,
  'Média Pesquisador': 3.708,
  'p-valor': '0,705'},
 {'IC95% difer': '-0.5-1.0',
  'Média Designer': 3.875,
  'Média Pesquisador': 3.792,
  'p-valor': '0,699'},
 {'IC95% difer': '-0.5-0.0',
  'Média Designer': 4.625,
  'Média Pesquisador': 4.875,
  'p-valor': '0,271'},
 {'IC95% difer': '-0.5-1.0',
  'Média Designer': 4.125,
  'Média Pesquisador': 3.792,
  'p-valor': '0,347'},
 {'IC95% difer': '-1.0-0.5',
  'Média Designer': 3.75,
  'Média Pesquisador': 3.875,
  'p-valor': '0,758'},
 {'IC95% difer': '-0.5-1.5',
  'Média Designer': 3.917,
  'Média Pesquisador': 3.333,
  'p-valor': '0,189'}]

In [134]:
data = pd.DataFrame(data=data, index=columns[:-3], columns=data[0].keys())
data.to_csv('total_pergunta_humam_wilcoxon.csv')
data

,Média Pesquisador,Média Designer,IC95% difer,p-valor
Dintinguíveis,3.833,3.833,-1.0-1.0,"0,924"
Proeminentes,3.708,3.875,-0.5-1.0,"0,705"
Eficiente,3.792,3.875,-0.5-1.0,"0,699"
Útil,4.875,4.625,-0.5-0.0,"0,271"
Estética,3.792,4.125,-0.5-1.0,"0,347"
Compreensão,3.875,3.750,-1.0-0.5,"0,758"
Interesse,3.333,3.917,-0.5-1.5,"0,189"


## Grau Graduação

In [135]:
graus = df_perg_quant.Grau.value_counts().index
printLis(graus)

-------------
-=< Lista >=-
-------------
0 -> Mestrado
1 -> Doutorado
2 -> Pós-Graduação
3 -> Graduação


In [136]:
data = []
for col in columns[:-3]:
        data.append(estates_table(
            list(df_perg_quant[(df_perg_quant.Origem == 'Pesquisador') & (df_perg_quant.Grau == graus[3])][col]),
            list(df_perg_quant[(df_perg_quant.Origem != 'Pesquisador') & (df_perg_quant.Grau == graus[3])].iloc[:-7][col])
        ))
data

/usr/local/lib/python3.7/dist-packages/scipy/stats/morestats.py:2879: UserWarning: Sample size too small for normal approximation.
  warnings.warn("Sample size too small for normal approximation.")


[{'IC95% difer': '0.0-1.5',
  'Média Designer': 4.158,
  'Média Pesquisador': 3.474,
  'p-valor': '0,094'},
 {'IC95% difer': '0.0-2.0',
  'Média Designer': 4.0,
  'Média Pesquisador': 2.947,
  'p-valor': '0,013'},
 {'IC95% difer': '-0.5-1.5',
  'Média Designer': 3.842,
  'Média Pesquisador': 3.316,
  'p-valor': '0,201'},
 {'IC95% difer': '0.0-0.0',
  'Média Designer': 4.895,
  'Média Pesquisador': 4.737,
  'p-valor': '0,276'},
 {'IC95% difer': '1.0-2.5',
  'Média Designer': 4.211,
  'Média Pesquisador': 2.526,
  'p-valor': '<0,001'},
 {'IC95% difer': '0.0-1.5',
  'Média Designer': 4.263,
  'Média Pesquisador': 3.526,
  'p-valor': '0,073'},
 {'IC95% difer': '0.5-2.0',
  'Média Designer': 3.789,
  'Média Pesquisador': 2.632,
  'p-valor': '0,007'}]

In [137]:
data = pd.DataFrame(data=data, index=columns[:-3], columns=data[0].keys())
data.to_csv('total_pergunta_gradua_wilcoxon.csv')
data

,Média Pesquisador,Média Designer,IC95% difer,p-valor
Dintinguíveis,3.474,4.158,0.0-1.5,"0,094"
Proeminentes,2.947,4.000,0.0-2.0,"0,013"
Eficiente,3.316,3.842,-0.5-1.5,"0,201"
Útil,4.737,4.895,0.0-0.0,"0,276"
Estética,2.526,4.211,1.0-2.5,"<0,001"
Compreensão,3.526,4.263,0.0-1.5,"0,073"
Interesse,2.632,3.789,0.5-2.0,"0,007"


## Grau Pós-Graduação

In [138]:
data = []
for col in columns[:-3]:
        data.append(estates_table(
            list(df_perg_quant[(df_perg_quant.Origem == 'Pesquisador') & (df_perg_quant.Grau == graus[2])][col]),
            list(df_perg_quant[(df_perg_quant.Origem != 'Pesquisador') & (df_perg_quant.Grau == graus[2])].iloc[:-9][col])
        ))
data

[{'IC95% difer': '-0.5-1.0',
  'Média Designer': 3.974,
  'Média Pesquisador': 3.737,
  'p-valor': '0,357'},
 {'IC95% difer': '0.0-1.0',
  'Média Designer': 4.0,
  'Média Pesquisador': 3.553,
  'p-valor': '0,105'},
 {'IC95% difer': '-0.5-0.5',
  'Média Designer': 3.868,
  'Média Pesquisador': 3.868,
  'p-valor': '0,973'},
 {'IC95% difer': '-0.5-0.0',
  'Média Designer': 4.632,
  'Média Pesquisador': 4.842,
  'p-valor': '0,268'},
 {'IC95% difer': '0.5-1.5',
  'Média Designer': 4.447,
  'Média Pesquisador': 3.237,
  'p-valor': '<0,001'},
 {'IC95% difer': '-0.5-0.5',
  'Média Designer': 3.868,
  'Média Pesquisador': 3.789,
  'p-valor': '0,731'},
 {'IC95% difer': '0.0-1.0',
  'Média Designer': 3.921,
  'Média Pesquisador': 3.395,
  'p-valor': '0,088'}]

In [139]:
data = pd.DataFrame(data=data, index=columns[:-3], columns=data[0].keys())
data.to_csv('total_pergunta_posGradua_wilcoxon.csv')
data

,Média Pesquisador,Média Designer,IC95% difer,p-valor
Dintinguíveis,3.737,3.974,-0.5-1.0,"0,357"
Proeminentes,3.553,4.000,0.0-1.0,"0,105"
Eficiente,3.868,3.868,-0.5-0.5,"0,973"
Útil,4.842,4.632,-0.5-0.0,"0,268"
Estética,3.237,4.447,0.5-1.5,"<0,001"
Compreensão,3.789,3.868,-0.5-0.5,"0,731"
Interesse,3.395,3.921,0.0-1.0,"0,088"


## Graus Mestrado

In [140]:

data = []
for col in columns[:-3]:
        data.append(estates_table(
            list(df_perg_quant[(df_perg_quant.Origem == 'Pesquisador') & (df_perg_quant.Grau == graus[0])].iloc[:-17][col]),
            list(df_perg_quant[(df_perg_quant.Origem != 'Pesquisador') & (df_perg_quant.Grau == graus[0])][col])
        ))
data

[{'IC95% difer': '0.0-1.0',
  'Média Designer': 4.129,
  'Média Pesquisador': 3.553,
  'p-valor': '0,002'},
 {'IC95% difer': '1.0-1.5',
  'Média Designer': 4.024,
  'Média Pesquisador': 2.988,
  'p-valor': '<0,001'},
 {'IC95% difer': '0.5-1.0',
  'Média Designer': 3.859,
  'Média Pesquisador': 3.212,
  'p-valor': '<0,001'},
 {'IC95% difer': '0.0-0.0',
  'Média Designer': 4.718,
  'Média Pesquisador': 4.506,
  'p-valor': '0,123'},
 {'IC95% difer': '1.0-2.0',
  'Média Designer': 4.282,
  'Média Pesquisador': 2.882,
  'p-valor': '<0,001'},
 {'IC95% difer': '0.5-1.0',
  'Média Designer': 3.976,
  'Média Pesquisador': 3.188,
  'p-valor': '<0,001'},
 {'IC95% difer': '1.0-2.0',
  'Média Designer': 4.153,
  'Média Pesquisador': 2.718,
  'p-valor': '<0,001'}]

In [141]:
data = pd.DataFrame(data=data, index=columns[:-3], columns=data[0].keys())
data.to_csv('total_pergunta_mestr_wilcoxon.csv')
data

,Média Pesquisador,Média Designer,IC95% difer,p-valor
Dintinguíveis,3.553,4.129,0.0-1.0,"0,002"
Proeminentes,2.988,4.024,1.0-1.5,"<0,001"
Eficiente,3.212,3.859,0.5-1.0,"<0,001"
Útil,4.506,4.718,0.0-0.0,"0,123"
Estética,2.882,4.282,1.0-2.0,"<0,001"
Compreensão,3.188,3.976,0.5-1.0,"<0,001"
Interesse,2.718,4.153,1.0-2.0,"<0,001"


## Grau Doutorado

In [142]:
data = []
for col in columns[:-3]:
        data.append(estates_table(
            list(df_perg_quant[(df_perg_quant.Origem == 'Pesquisador') & (df_perg_quant.Grau == graus[1])][col]),
            list(df_perg_quant[(df_perg_quant.Origem != 'Pesquisador') & (df_perg_quant.Grau == graus[1])].iloc[:-1][col])
        ))
data

[{'IC95% difer': '0.0-1.0',
  'Média Designer': 4.013,
  'Média Pesquisador': 3.364,
  'p-valor': '0,005'},
 {'IC95% difer': '0.5-1.5',
  'Média Designer': 4.026,
  'Média Pesquisador': 3.065,
  'p-valor': '<0,001'},
 {'IC95% difer': '0.5-1.5',
  'Média Designer': 3.805,
  'Média Pesquisador': 2.987,
  'p-valor': '0,002'},
 {'IC95% difer': '0.0-0.5',
  'Média Designer': 4.714,
  'Média Pesquisador': 4.26,
  'p-valor': '0,014'},
 {'IC95% difer': '1.5-2.0',
  'Média Designer': 4.286,
  'Média Pesquisador': 2.558,
  'p-valor': '<0,001'},
 {'IC95% difer': '0.5-1.5',
  'Média Designer': 3.857,
  'Média Pesquisador': 3.013,
  'p-valor': '<0,001'},
 {'IC95% difer': '0.5-1.5',
  'Média Designer': 4.0,
  'Média Pesquisador': 3.013,
  'p-valor': '<0,001'}]

In [143]:
data = pd.DataFrame(data=data, index=columns[:-3], columns=data[0].keys())
data.to_csv('total_pergunta_doc_wilcoxon.csv')
data

,Média Pesquisador,Média Designer,IC95% difer,p-valor
Dintinguíveis,3.364,4.013,0.0-1.0,"0,005"
Proeminentes,3.065,4.026,0.5-1.5,"<0,001"
Eficiente,2.987,3.805,0.5-1.5,"0,002"
Útil,4.260,4.714,0.0-0.5,"0,014"
Estética,2.558,4.286,1.5-2.0,"<0,001"
Compreensão,3.013,3.857,0.5-1.5,"<0,001"
Interesse,3.013,4.000,0.5-1.5,"<0,001"


# Exportar para pdf

In [149]:
!python --version

Python 3.7.13


In [150]:
!pip freeze

absl-py==1.0.0
alabaster==0.7.12
albumentations==0.1.12
altair==4.2.0
appdirs==1.4.4
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
arviz==0.11.4
astor==0.8.1
astropy==4.3.1
astunparse==1.6.3
atari-py==0.2.9
atomicwrites==1.4.0
attrs==21.4.0
audioread==2.1.9
autograd==1.3
Babel==2.9.1
backcall==0.2.0
beautifulsoup4==4.6.3
bleach==4.1.0
blis==0.4.1
bokeh==2.3.3
Bottleneck==1.3.4
branca==0.4.2
bs4==0.0.1
CacheControl==0.12.10
cached-property==1.5.2
cachetools==4.2.4
catalogue==1.0.0
certifi==2021.10.8
cffi==1.15.0
cftime==1.6.0
chardet==3.0.4
charset-normalizer==2.0.12
click==7.1.2
cloudpickle==1.3.0
cmake==3.12.0
cmdstanpy==0.9.5
colorcet==3.0.0
colorlover==0.3.0
community==1.0.0b1
contextlib2==0.5.5
convertdate==2.4.0
coverage==3.7.1
coveralls==0.5
crcmod==1.7
cufflinks==0.17.3
cvxopt==1.2.7
cvxpy==1.0.31
cycler==0.11.0
cymem==2.0.6
Cython==0.29.28
daft==0.0.4
dask==2.12.0
datascience==0.10.6
debugpy==1.0.0
decorator==4.4.2
defusedxml==0.7.1
descartes==1.1.0
dill==0.3.4
distributed==

In [151]:
## Conectar ao Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [154]:
## Instalar Text-Line
# % cd drive/MyDrive/Colab\ Notebooks
# !sudo apt-get update
# !sudo apt-get install texlive-full -y

In [155]:
## Exportar para PDF
!jupyter nbconvert --to pdf 04-Wilcoxon_analysis_2022-03-28.ipynb

[NbConvertApp] Converting notebook 04-Wilcoxon_analysis_2022-03-28.ipynb to pdf
[NbConvertApp] Writing 92422 bytes to ./notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: ['xelatex', './notebook.tex', '-quiet']
[NbConvertApp] Running bibtex 1 time: ['bibtex', './notebook']
[NbConvertApp] WARNING | bibtex had problems, most likely because there were no citations
[NbConvertApp] PDF successfully created
[NbConvertApp] Writing 90067 bytes to 04-Wilcoxon_analysis_2022-03-28.pdf
